In [2]:
import nltk, re
from nltk.stem.porter import PorterStemmer
import pandas as pd



# Reference: https://ijbnpa.biomedcentral.com/articles/10.1186/s12966-017-0525-8

# Try: need to have at least 1 key verb and 1 key noun to be classified as sedentary
# instead of relying on just 1 keyword


# French translator
from googletrans import Translator
translator = Translator()

# stemmer
porter = nltk.PorterStemmer()



In [3]:
# test set
test = pd.read_json('~/Documents/PhD/HBDC/FilteredTweets/_tweets_2017.12.01.20.09.json', lines=True)

test

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
0,NaN,None,2017-10-29 22:43:39,"[0, 12]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.245877e+17,9.245877e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",u inspire me https://t.co/VdECceNqpI,2017-10-29 22:43:39.375,False,"{u'follow_request_sent': None, u'profile_use_b..."
1,NaN,None,2017-10-29 22:43:39,"[0, 26]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.247346e+17,9.247346e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",An icon of Canadian art... https://t.co/Fxk3w5...,2017-10-29 22:43:39.431,False,"{u'follow_request_sent': None, u'profile_use_b..."
2,NaN,None,2017-10-29 22:43:40,"[9, 32]","{u'user_mentions': [{u'id': 203123011, u'indic...",NaN,NaN,0,False,low,...,9.244376e+17,9.244376e+17,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@CTVNews https://t.co/vcQUwe5ww6,2017-10-29 22:43:40.198,False,"{u'follow_request_sent': None, u'profile_use_b..."
3,NaN,None,2017-10-29 22:43:40,"[40, 49]","{u'user_mentions': [{u'id': 747981270, u'indic...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@rouxx21 @TalonFruitBat @Twilight_Saint &lt;33333,2017-10-29 22:43:40.311,False,"{u'follow_request_sent': None, u'profile_use_b..."
4,NaN,None,2017-10-29 22:43:39,"[0, 62]","{u'user_mentions': [{u'id': 1565023633, u'indi...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",I get A's &amp; B's and she gets D's....nuts!!...,2017-10-29 22:43:39.434,False,"{u'follow_request_sent': None, u'profile_use_b..."
5,NaN,None,2017-10-29 22:43:40,"[27, 140]","{u'user_mentions': [{u'id': 4560764232, u'indi...",NaN,"{u'display_text_range': [27, 167], u'entities'...",0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@_Nick_Nation_ @nickcarter Can believe its bee...,2017-10-29 22:43:40.770,True,"{u'follow_request_sent': None, u'profile_use_b..."
6,NaN,None,2017-10-29 22:43:40,"[0, 100]","{u'user_mentions': [{u'id': 567010052, u'indic...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Fav song 🔥 Honestly I’ve been playing this so...,2017-10-29 22:43:40.808,False,"{u'follow_request_sent': None, u'profile_use_b..."
7,NaN,None,2017-10-29 22:43:41,"[0, 0]","{u'user_mentions': [], u'symbols': [], u'hasht...",{u'media': [{u'expanded_url': u'https://twitte...,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",https://t.co/dq4Ti7e1wx,2017-10-29 22:43:41.667,False,"{u'follow_request_sent': None, u'profile_use_b..."
8,NaN,"{u'type': u'Point', u'coordinates': [-66.0196,...",2017-10-29 22:43:42,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""https://untappd.com"" rel=""nofollow"">U...",It has more sweet flavour notes than most othe...,2017-10-29 22:43:42.172,False,"{u'follow_request_sent': None, u'profile_use_b..."
9,NaN,None,2017-10-29 22:43:42,"[0, 18]","{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,9.247512e+17,9.247512e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Correct.That’s it. https://t.co/0jtu2oILGN,2017-10-29 22:43:42.359,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [4]:
# Verbs indicate sedentary
sedentary_verbs = ['watch', 'chill', 'work', 'study', 'lying', 'recline', 'sit\s', 'read\s', 'draw', 'paint'
        , 'drive']

# Nouns indicate sedentary
sedentary_nouns = ['game', 'video game', 'school', 'office', 'television', 'tv', 
         'computer', 'phone', 'iphone', 'tablet', 'ipad', 'kindle', 'book', 
         'newspaper', 'homework', 'car\s', 'plane', 'bus', 'train', 'netflix',
         'bed', 'the play', 'the show'] 



# list of sport leagues
sedentary_leagues = ['nba', 'worldseri', 'nfl', 'mls', 'baseball', 'mlb', 'soccer'
                     , 'mls', 'ufc', 'nhl', 'cfl']

sedentary_exclusion = ['look for work', 'hire', 'job', 'career']



sedentary_verbs_fr = [translator.translate(x, dest='fr').text for x in sedentary_verbs]
#sedentary_verbs_fr

sedentary_nouns_fr = [translator.translate(x, dest='fr').text for x in sedentary_nouns]
#sedentary_nouns_fr  



sedentary_verbs_stemmed = [porter.stem(item) for item in sedentary_verbs]
#sedentary_verbs_stemmed

# stemming nouns doesnt make sense
#sedentary_nouns_stemmed = [porter.stem(item) for item in sedentary_nouns]
#sedentary_verbs_stemmed




# Regex pattern
sedentary_verbs_regex = "\s(" + '|'.join(sedentary_verbs + sedentary_verbs_stemmed + sedentary_verbs_fr) + ')'



sedentary_nouns_regex = "\s(" + '|'.join(sedentary_nouns + sedentary_nouns_fr + sedentary_leagues) + ')'
#sedentary_nouns_regex = '\s' + '|'.join(sedentary_nouns + sedentary_nouns_stemmed)
#sedentary_nouns_regex = re.sub(' ', '\ ', sedentary_nouns_regex)


sedentary_exclusion_regex = "\s(" + '|'.join(sedentary_exclusion) + ')'




In [5]:
##### Physical Activity
# Verbs indicate Phyiscal Activity
activity_verbs = ['play', 'train', 'run\s', 'hike', 'ski\s', 'bike', 'lift']

# Nouns indicate Physical Activity
activity_nouns = [ 'gym\s', 'workout', 'work\ out', 'basketball', 'basket\ ball'
                  ,'hockey', 'biking', 'bb', 'bball', 'baseball', 'climb', 'dance', 'dancin'
                  ,'skate', 'skating', 'squat', 'weights', 'gains', 'volleyball', 'yoga'
                 , 'park', 'mountain'] 


# combine nouns with wikipedia sports
#FIX THE SPACE IN BETWEEN WORDS WITH \(space)
with open('sportslist.txt') as f:
    lines = f.read().splitlines()

    activity_nouns = activity_nouns + [x.lower() for x in lines if x.lower() not in activity_nouns] # removed 3 duplicates

# Exclusion
activity_exclusion = ['look for work', 'hire', 'job', 'career', 'game']


# # French
activity_verbs_fr = [translator.translate(x, dest='fr').text for x in activity_verbs]

with open('sportslist_fr.txt') as f:# too many to make google calls
    lines = f.read().splitlines()
activity_nouns_fr = [x.lower() for x in lines if x.lower() not in activity_nouns]


# stemmed form
activity_verbs_stemmed = [porter.stem(item) for item in activity_verbs]

# Regex pattern
activity_verbs_regex = "\s(" + '|'.join(activity_verbs + activity_verbs_stemmed + activity_verbs_fr) + ')'


activity_nouns_regex = "\s(" + '|'.join(activity_nouns + activity_nouns_fr) + ')'


activity_exclusion_regex = "\s(" + '|'.join(activity_exclusion) + ')'


temp = re.sub('a.k.a','',activity_nouns_regex)
temp = re.sub('\.','', temp)
temp = re.sub('/','',temp)
temp = re.sub('\[','',temp)
temp = re.sub('\]','',temp)
temp = re.sub('|game','',temp)
#print temp

activity_nouns_regex = temp


In [6]:
##### Sleep
# Verbs indicate Phyiscal Activity
sleep_verbs = ['sleep', 'nap\s', 'rest\s', 'zzz.?\s', 'pass out', 'get up', 'wake up'
               , 'asleep']

# Nouns indicate Physical Activity
sleep_nouns = ['bed', 'night', 'last night', 'today', 'sack', 'morning', 'insomnia'] 
# Exclusion
sleep_exclusion = ['restaurant']


# # French
sleep_verbs_fr = [translator.translate(x, dest='fr').text for x in sleep_verbs]
sleep_nouns_fr = [translator.translate(x, dest='fr').text for x in sleep_nouns]

# stemmed form
sleep_verbs_stemmed = [porter.stem(item) for item in sleep_verbs]

# Regex pattern
sleep_verbs_regex = "\s(" + '|'.join(sleep_verbs + sleep_verbs_stemmed + sleep_verbs_fr) + ')'


sleep_nouns_regex = "\s(" + '|'.join(sleep_nouns + sleep_nouns_fr) + ')'


sleep_exclusion_regex = "\s(" + '|'.join(sleep_exclusion) + ')'





In [8]:
# tokenize each tweets
# look for match in keyword
# have to include word boundary before, and some need after as well

for i, item in enumerate(test.text):
    #print item 
    lowered = item.lower()
    
    tokens = nltk.word_tokenize(lowered)

    stemmed_tokens = [porter.stem(x) for x in tokens]

    #print stemmed_tokens
    
    ##### Sedentary 
    sedentary_verbs_count = len(re.findall(sedentary_verbs_regex, ' '.join(stemmed_tokens)))
    #print re.findall(verbs_regex, ' '.join(stemmed_tokens))
    #print verbs_count

    sedentary_noun_count = len(re.findall(sedentary_nouns_regex, ' '.join(stemmed_tokens)))
    #print re.findall(nouns_regex, ' '.join(stemmed_tokens))
    #print noun_count
    #print '\n'
    
    sedentary_exlusion_count = len(re.findall(sedentary_exclusion_regex, ' '.join(stemmed_tokens)))

    if sedentary_verbs_count >=1 and sedentary_noun_count >=1 and sedentary_exlusion_count == 0:
    	test.at[i, 'sedentary'] = 1
    	print 'sedentary'
        print item
    	#print stemmed_tokens
    	print 'verb: ', re.findall(sedentary_verbs_regex, ' '.join(stemmed_tokens))
    	print 'noun: ', re.findall(sedentary_nouns_regex, ' '.join(stemmed_tokens))
        print 'exclude', re.findall(sedentary_exclusion_regex, ' '.join(stemmed_tokens))
    	print '\n'
    else:
    	test.at[i, 'sedentary'] = 0

    
    ##### Phyiscal Activity 
    activity_verbs_count = len(re.findall(activity_verbs_regex, ' '.join(stemmed_tokens)))

    activity_noun_count = len(re.findall(activity_nouns_regex, ' '.join(stemmed_tokens)))
 
    activity_exlusion_count = len(re.findall(activity_exclusion_regex, ' '.join(stemmed_tokens)))

    if activity_verbs_count >=1 and activity_noun_count >=1 and activity_exlusion_count == 0:
    	test.at[i, 'activity'] = 1
    	print 'activity'
        print item
    	#print stemmed_tokens
    	print 'verb: ',  re.findall(activity_verbs_regex, ' '.join(stemmed_tokens))
    	print 'noun: ', re.findall(activity_nouns_regex, ' '.join(stemmed_tokens))
        print 'exclude; ', re.findall(sedentary_exclusion_regex, ' '.join(stemmed_tokens))
    	print '\n'
    else:
    	test.at[i, 'activity'] = 0      
        
    ##### Phyiscal Activity 
    sleep_verbs_count = len(re.findall(sleep_verbs_regex, ' '.join(stemmed_tokens)))

    sleep_noun_count = len(re.findall(sleep_nouns_regex, ' '.join(stemmed_tokens)))
 
    sleep_exlusion_count = len(re.findall(sleep_exclusion_regex, ' '.join(stemmed_tokens)))

    if sleep_verbs_count >=1 and sleep_noun_count >=1 and sleep_exlusion_count == 0:
    	test.at[i, 'sleep'] = 1
    	print 'sleep'
        print item
    	#print stemmed_tokens
    	print 'verb: ',  re.findall(sleep_verbs_regex, ' '.join(stemmed_tokens))
    	print 'noun: ', re.findall(sleep_nouns_regex, ' '.join(stemmed_tokens))
        print 'exclude; ', re.findall(sleep_exclusion_regex, ' '.join(stemmed_tokens))
    	print '\n'
    else:
    	test.at[i, 'sleep'] = 0         
        
        
        
        

activity
TNT Fitness and LA got to play with a Strong Board today. A Strong Board is like a BOSU on… https://t.co/UBvOtctYzo
verb:  [u'play']
noun:  [u'go']
exclude;  []


sedentary
Even though I slept from 8pm to 530 am and had a nap after work, I'm still exhausted and I'm going back to bed now… https://t.co/PpL1bztSny
verb:  [u'work']
noun:  [u'bed']
exclude []


sleep
Even though I slept from 8pm to 530 am and had a nap after work, I'm still exhausted and I'm going back to bed now… https://t.co/PpL1bztSny
verb:  [u'nap ']
noun:  [u'bed']
exclude;  []


sedentary
Okie dokie... Now that I’m just lying in bed; refusing to move because I’m warm as fuck and wrapped like a burrito.… https://t.co/I0RGGJXa7s
verb:  [u'lie']
noun:  [u'bed']
exclude []


activity
@TheFortressTurf @cranky_titz69 Muddin' runs in that family.
verb:  [u'run ']
noun:  [u'cran']
exclude;  []


sedentary
Everytime I drive my little sis somewhere she always wants to listen to Migos
verb:  [u'drive']
noun:  [u'lit']
e

noun:  [u'nhl']
exclude []


activity
It’s the same story over n over again with this team! Keeps saying they gotta be better n other shit but somehow they r playing worse!
verb:  [u'play']
noun:  [u'go', u'other']
exclude;  []


activity
Matt Murray is a winning playoff goalie, regular season well....... 😅😅😅
verb:  [u'play']
noun:  [u'go']
exclude;  []


activity
Trail run today &amp; did a few Kms on grass2work on ankle strength 👍🏼followed up with abit of time@the gym this evenin… https://t.co/IcX0JLSuex
verb:  [u'run ']
noun:  [u'gym ']
exclude;  []


sedentary
My workout...1 sit-up... getting out of bed...when my alarm goes off...1 stretch...to turn my coffee maker on...
verb:  [u'work']
noun:  [u'bed']
exclude []


activity
Oh and there was the one kid I used to play baseball with who got stoned and drove an ATV into a barbed wire fence and died.
verb:  [u'play']
noun:  [u'go']
exclude;  []


sedentary
@Snoopy @ABCNetwork I Have it Recording So I Can Watch it Before Bed
verb:  [u'

noun:  [u'netflix']
exclude []


activity
My #Knicks are giving me cold sweats tonight. Guys are playing goooooddddd
verb:  [u'play']
noun:  [u'go']
exclude;  []


sedentary
What to watch tonight?!?  #worldseries #steelers #whitecaps #thewalkingdead @ Richmond Hill,… https://t.co/T36Ajq9Q8C
verb:  [u'watch']
noun:  [u'worldseri']
exclude []


activity
One of the tracks from the Evangelion soundtrack plays in Shin Gojira! https://t.co/nGtNdJJ1F5
verb:  [u'play']
noun:  [u'go']
exclude;  []


sedentary
My poor car driving over this. Thankfully I was going slow!!!! @CityNews @cityoftoronto https://t.co/LQnfC1nNeF
verb:  [u'drive']
noun:  [u'car ']
exclude []


activity
@Mike_P_Williams @Nortonvt @ POTUS what we should have done is ensure you never got the RNC ticket to run for President!
verb:  [u'run ']
noun:  [u'go']
exclude;  []


activity
It's almost impossible to be this consistently bad at running the ball. FML 

#ONEPRIDE
verb:  [u'run ']
noun:  [u'ball']
exclude;  []


activity
.@

Wish I was watching this game
verb:  [u'watch']
noun:  [u'game']
exclude []


activity
When I get married I'm having a dowry.. Eff that. Run my family a goat and an Ox.  #90dayfiance
verb:  [u'run ']
noun:  [u'go']
exclude;  []


activity
Joe Buck climbed right on the Gurriel excuse train, didn't be? Rookie, preparing for big leagues, etc.
verb:  [u'train']
noun:  [u'climb']
exclude;  []


activity
Joe Buck climbed right on the Gurriel excuse train, didn't he? Rookie, preparing for big leagues, etc.
verb:  [u'train']
noun:  [u'climb']
exclude;  []


activity
@thedanield That's your stolen bike being sold.
verb:  [u'bike']
noun:  [u'sol']
exclude;  []


activity
My phone is fucked lol and got the run around with #Rodgers today.
verb:  [u'run ']
noun:  [u'go']
exclude;  []


activity
Way to go @el_yuly10 🍍🍍🍍🍍🍍🍍🍍🔥🔥🍍🍍🍍🍍🍍🍍🍍🍍. That home run was a no doubter.
verb:  [u'run ']
noun:  [u'go']
exclude;  []


sedentary
Holy crap it’s 10:11 and it’s only the end of the 4th.    I have to work tomor

noun:  [u'worldseri']
exclude []


activity
That boy @TeamJuJu got his bike back! Now y’all in trouble🙌🏾😂 #CBME
verb:  [u'bike']
noun:  [u'go']
exclude;  []


activity
Cooter explain to me running to the half yard mark on first down and then passing on 2nd down??? Run the F'ing ball!!!! Idiot
verb:  [u'run ', u'run ']
noun:  [u'ball']
exclude;  []


activity
Unless Prater's leg fell off &amp; nobody knew, Caldwell should have taken the 3. Play calling &amp; personnel not good enough on goal line for a TD
verb:  [u'play']
noun:  [u'go', u'go']
exclude;  []


sedentary
@ScottHelman I have not watched the show.
verb:  [u'watch']
noun:  [u'the show']
exclude []


activity
Tree grate parkette #landarch Literally running into claudecormierlandscape projects everywhere… https://t.co/FBByq0sv8G
verb:  [u'run ']
noun:  [u'park']
exclude;  []


activity
The other thing i love here is the fast INTERNET. Nobody knows how to play dota hikhik😝😝😝😝
verb:  [u'play']
noun:  [u'other']
exclude;  []


sle

...

noun:  [u'go']
exclude;  []


sedentary
Do u ever just sit in ur car for a good half hour bc ur too lazy to get out bc same
verb:  [u'sit ']
noun:  [u'car ']
exclude []


sedentary
So looking forward to my next day off.. all I wanna do is lay in bed and watch #StrangersThings2
verb:  [u'watch']
noun:  [u'bed']
exclude []


sedentary
#studioapartment perks - being able to watch tv from bed, the kitchen, and the bathroom (door… https://t.co/Fb6OWHWZeT
verb:  [u'studi', u'watch']
noun:  [u'tv', u'bed']
exclude []


sedentary
@SarahOttawa Is your daughter going into the trades, Sarah? Maybe our girls could start a business doing work for d… https://t.co/GZK5aaV3Q2
verb:  [u'work']
noun:  [u'bus']
exclude []


activity
So far, the Artist Development Program students have gone through hands on training with… https://t.co/CIj8s9jsXS
verb:  [u'train']
noun:  [u'go']
exclude;  []


activity
I'm sorry but if you take rec hockey serious, don't play lmao
verb:  [u'play']
noun:  [u'hockey']
exclude

verb:  [u'sleep']
noun:  [u'night']
exclude;  []


sedentary
“Some may see immortality as a gift,” said no one who has ever read a book or seen a movie. #InnerSpace #BladeofTheImmortal
verb:  [u'read ']
noun:  [u'book']
exclude []


activity
Why do #Cavs always play like a shit? Please #CavsNation when are we going to start winning
verb:  [u'play']
noun:  [u'go']
exclude;  []


activity
Can’t thank @KitchFire enough for giving myself &amp;all the other women this opportunity to train together! Being out… https://t.co/6dANN8fjp8
verb:  [u'train']
noun:  [u'other']
exclude;  []


sleep
Sweet dreams 😴
—
#bedtime #sleeptight #goodnight #SleepBetterLiveBetter… https://t.co/Aqj7P7XwxL
verb:  [u'sleep', u'sleep']
noun:  [u'bed']
exclude;  []


activity
@lmorchard I'm going for one. We can be the cool kids in the back row playing Bubble Bobble.
verb:  [u'play']
noun:  [u'go']
exclude;  []


sleep
Help me fundraise for homeless youth!
Tomorrow night's forecast is 5°c and rain.
I'll be sleeping 

noun:  [u'netflix']
exclude []


activity
2 very good young players. https://t.co/StbQfmeBZT
verb:  [u'play']
noun:  [u'go']
exclude;  []


activity
Amazon Prime Music: Over 1 million songs.

Google Play Music: Over 35 million songs.

Prime Music truly is a bolt-on service...
verb:  [u'play']
noun:  [u'go']
exclude;  []


activity
🚨O’Connell🚨 @RyannOconnell gets in on the act and it’s 3-0 Vees! It’s another power play goal as well! O’Connell’s… https://t.co/AU4nadg9qa
verb:  [u'play']
noun:  [u'go']
exclude;  []


activity
.@Doc_Pow really wants us to skip work tomorrow and go skiing. I'm in, u? #HootupYVR
verb:  [u'ski ']
noun:  [u'go']
exclude;  []


activity
3 more days for the new #GOALKING #Goalkeeper camp for youth players in the #Edmonton @yeg region. Get registered n… https://t.co/ehcKGpoEE3
verb:  [u'play']
noun:  [u'go', u'go']
exclude;  []


activity
Hey @ITMovieOfficial can we make grown up #EddieKaspbrak trans &amp; can #KristenWiig play her? Now that wud b an awesome #cas

noun:  [u'hope']
exclude;  []


sedentary
No Cars Go by Arcade Fire is #nowplaying in Vera's On The Drive, Vancouver.
verb:  [u'drive']
noun:  [u'car ']
exclude []


activity
So i got myself a new dentist but then they played country music in their lobby and I never went back. I hate cowboy culture.
verb:  [u'play']
noun:  [u'go', u'cowboy']
exclude;  []


activity
I love hockey players https://t.co/iSsEAsIfGE
verb:  [u'play']
noun:  [u'hockey']
exclude;  []


activity
.@blazerhockey within one on Chyzowski power play goal
verb:  [u'play']
noun:  [u'go']
exclude;  []


sleep
@Titus88Titus @Lily_Carver_ it was on tcm a few nights ago. i fell asleep though.   #WestWed
verb:  [u'asleep']
noun:  [u'night']
exclude;  []


sedentary
I need someone to doodle something like a character and DM them to me cause I literally have no ideas to draw. Thanks ;;_;;
verb:  [u'draw']
noun:  [u'lit']
exclude []


sleep
In case you want to go to sleep knowing you've learned something today. 
#lol #bcwine… 

noun:  [u'book']
exclude []


sedentary
I've been watching naruto all night lol, my little brother put me on omgg
verb:  [u'watch']
noun:  [u'lit']
exclude []


sedentary
Buys a ticket to anywhere for someone who drives a fast car https://t.co/RP9KBvFdeR
verb:  [u'drive']
noun:  [u'car ']
exclude []


sleep
I’d rather not wake up at exactly 3 am every night, you know, for reasons and stuff...
verb:  [u'wake up']
noun:  [u'night']
exclude;  []


sedentary
@VancityBuzz Let it be. Point 2 - Game Day Traffic! Driving north on a game/event night can already be problematic… https://t.co/0h4YGm4Jtx
verb:  [u'drive']
noun:  [u'game', u'game']
exclude []


activity
@alibajwa I think Canada can play a meaningful role in such seat in comparison to other countries. We have been pro… https://t.co/7lioz0Hy5n
verb:  [u'play']
noun:  [u'other']
exclude;  []


activity
Dear @pizzafreshslice, why was American Psycho playing in one of your stores? Thankfully no kids had to see this! 👀… https://t.co/BApvs